In [3]:
import numpy as np
import pandas as pd

train_labels = np.load('preprocessed_files/train_labels.npy')
test_labels = np.load('preprocessed_files/test_labels.npy')

train_images_untouched = pd.read_pickle('preprocessed_files/train_images_untouched.pkl')
test_images_untouched = pd.read_pickle('preprocessed_files/test_images_untouched.pkl')

train_images_touched = pd.read_pickle('preprocessed_files/train_images_touched.pkl')
test_images_touched = pd.read_pickle('preprocessed_files/test_images_touched.pkl')

In [50]:
def get_means_stds_ProbabsOfLabels(df):
    dfs = {}
    probs = []
    for i in range(10):
            dfs["df{0}".format(i)] = df[train_labels == i]
            probs.append(len(df[train_labels == i]) / len(df))
    means = []
    stds = []
    for key in dfs:
        pixels = list(dfs[key])
        pixel_means = []
        pixel_stds = []
        for pixel in pixels:
            pixel_means.append(np.mean(dfs[key][pixel]))
            pixel_stds.append(np.std(dfs[key][pixel]))
        means.append(pixel_means)
        stds.append(pixel_stds)
    return means, stds, probs


In [84]:
means_untouched, stds_untouched, probs_untouched = get_means_stds_ProbabsOfLabels(train_images_untouched)
means_touched, stds_touched, probs_touched = get_means_stds_ProbabsOfLabels(train_images_touched)

In [107]:
means_untouched_images = []
for flat_image in means_untouched:
    means_untouched_images.append(np.array(flat_image).reshape(28,28))

In [126]:
from PIL import Image
for i in range(len(means_untouched_images)):
    im = Image.fromarray(means_untouched_images[i])
    im = im.convert("L")
#     path=("image_")
    im.save("image_" + str(i) + ".png")
im.save('image.bmp')

In [72]:
def normal(xi,mean,std):
    if std == 0:
        return 0
    else:
        return 1 / (2 * np.pi * std**2)**0.5 * np.e**-((xi - mean)**2 / (2*std**2))

In [95]:
def classifier(x,means,stds,probs):
    total_probabilities = np.zeros(10)
    for i in range(len(total_probabilities)):
        total_probabilities[i] += np.log(probs[i])
        for j in range(len(x)):
            n = normal(x[j], means[i][j], stds[i][j])
            if n > 0:
                total_probabilities[i] += np.log(n)
    return np.argmax(total_probabilities)

In [100]:
train_untouched_predictions = []
for index, row in train_images_untouched.iterrows():
    train_untouched_predictions.append(classifier(row, means_untouched, stds_untouched, probs_untouched))
    if index % 1000 == 0:
            print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000


In [101]:
train_touched_predictions = []
for index, row in train_images_touched.iterrows():
    train_touched_predictions.append(classifier(row, means_touched, stds_touched, probs_touched))
    if index % 1000 == 0:
            print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000


In [92]:
test_untouched_predictions = []
for index, row in test_images_untouched.iterrows():
    test_untouched_predictions.append(classifier(row, means_untouched, stds_untouched, probs_untouched))
    if index % 1000 == 0:
            print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [86]:
test_touched_predictions = []
for index, row in test_images_touched.iterrows():
    test_touched_predictions.append(classifier(row, means_touched, stds_touched, probs_touched))
    if index % 1000 == 0:
            print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [112]:
untouched_right = 0
touched_right = 0
for i in range(len(touched_predictions)):
    if train_touched_predictions[i] == train_labels[i]:
        touched_right += 1
    
    if train_untouched_predictions[i] == train_labels[i]:
        untouched_right += 1

train_untouched_accuracy = untouched_right / len(untouched_predictions)
train_touched_accuracy = touched_right / len(touched_predictions)
print('train_untouched_accuracy', train_untouched_accuracy)
print('train_touched_accuracy', train_touched_accuracy)


train_untouched_accuracy 0.6215
train_touched_accuracy 0.823


In [98]:
untouched_right = 0
touched_right = 0
for i in range(len(touched_predictions)):
    if test_touched_predictions[i] == test_labels[i]:
        touched_right += 1
    
    if test_untouched_predictions[i] == test_labels[i]:
        untouched_right += 1

test_untouched_accuracy = untouched_right / len(untouched_predictions)
test_touched_accuracy = touched_right / len(touched_predictions)
print('test_untouched_accuracy', test_untouched_accuracy)
print('test_touched_accuracy', test_touched_accuracy)


test_untouched_accuracy 0.6168
test_touched_accuracy 0.8287
